In [5]:
import torch

from maskrcnn_benchmark.structures.bounding_box import BoxList

box_data = torch.rand(3, 4)
box_data

tensor([[0.1250, 0.0263, 0.6050, 0.6135],
        [0.1619, 0.5308, 0.8323, 0.8469],
        [0.7332, 0.8992, 0.0204, 0.4014]])

In [7]:

box = BoxList(box_data, (300, 300))
box

BoxList(num_boxes=3, image_width=300, image_height=300, mode=xyxy)

In [10]:
box_data_subset = box.bbox[[True, False, True]]
box_data_subset

tensor([[0.1250, 0.0263, 0.6050, 0.6135],
        [0.7332, 0.8992, 0.0204, 0.4014]])

In [21]:
b = box.bbox
b[1] = torch.tensor([1, 2, 3, 5])

In [22]:
box.bbox

tensor([[0.1250, 0.0263, 0.6050, 0.6135],
        [1.0000, 2.0000, 3.0000, 5.0000],
        [0.7332, 0.8992, 0.0204, 0.4014]])

In [19]:
box_data.clone()

tensor([[0.1250, 0.0263, 0.6050, 0.6135],
        [1.0000, 2.0000, 3.0000, 4.0000],
        [0.7332, 0.8992, 0.0204, 0.4014]])